In [26]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
import ast #convert columns wiht lists to lst
import os
#set enviorment
os.chdir(r"C:\Users\mese4\Documents\The Data incubator\project\2020-08-05\cache")

#open metadata into pandas df
df = pd.read_csv('df_mesminer_final.csv', encoding='ISO-8859-1',index_col=0)
df['Biomedical_Entities'] =  df['Biomedical_Entities'].apply(ast.literal_eval)
#df['Genes'] =  df['Genes'].apply(ast.literal_eval)
#df['Drugs'] =  df['Drugs'].apply(ast.literal_eval)



In [27]:
df.head(2)

,index,cord_uid,publish_time,preprint,corpus,Genes,Drugs,Biomedical_Entities,Genes_Upregulated,Genes_Downregulated,Genes_Nonregulated
0,0,p58n51xv,2019-12-05,Peer-Review,Crandell-Rees Feline Kidney Cell (CRFK) immort...,"['s100a4', 'vim', 'vwf', 's100a4']",[],"[Crandell-Rees Feline Kidney Cell, CRFK, immor...",[],[],"[S100A4, vimentin, vWF, S100A4]"
1,1,heo6mtxk,2019-12-05,Peer-Review,African swine fever transmissible animal consu...,[],[],"[African swine fever, transmissible, animal co...",[],[],[]


In [28]:
#you can create a search of papers that contain keywords, add them in the list
selection = ['liver']
df = df[pd.DataFrame(df.Biomedical_Entities.tolist()).isin(selection).any(1)]

#you can create only look at preprints or peer reviewed papres
#selection = ['Peer-Review']
#df = df[pd.DataFrame(df.Preprint.tolist()).isin(selection).any(1)]

df

,index,cord_uid,publish_time,preprint,corpus,Genes,Drugs,Biomedical_Entities,Genes_Upregulated,Genes_Downregulated,Genes_Nonregulated
55,55,6vo1lmne,2020-01-06,Peer-Review,BACKGROUND: Growth hormone inducible transmemb...,"['ghitm', 'ghitm', 'ghitm', 'ghitm', 'ghitm', ...",[],"[BACKGROUND, Growth hormone, GHITM, transmembr...",[],[],[Growth hormone inducible transmembrane protei...
176,176,d6domrgx,2020-02-03,Peer-Review,Worldwide strategy 2010 2017 aimed controlling...,[],[],"[controlling, malarial, parasite, Plasmodium f...",[],[],[]
202,202,wd2zkgbt,2020-01-13,Peer-Review,ability preserve metabolically active liver ex...,[],"['dextrose, unspecified form', 'atp']","[ability, metabolically, active liver, ex vivo...",[],[],[]
562,562,dvhkk405,2020-03-14,Peer-Review,chronic low-grade inflammation adipose tissue ...,"['ins', 'ins', 'prkaa2', 'prkaa2', 'prkaa2']","['dextrose, unspecified form']","[chronic low-grade, inflammation, adipose tiss...","[insulin, insulin, AMPK]",[],[]
605,605,90wgsxhs,2019-12-27,Peer-Review,BACKGROUND & AIMS: Inflammation play important...,"['casp1', 'nlrp1', 'nlrp3', 'mrc1', 'cd163']",[],"[AIMS, Inflammation, pathogenesis, cholestatic...",[NLRP1],[],"[Caspase-1, NLRP3, CD206, CD163]"
...,...,...,...,...,...,...,...,...,...,...,...
71035,1054,g703hhm4,11/07/2020,Peer-Review,order investigate enterobacteria presence invo...,[],[],"[investigate, enterobacteria, presence, second...",[],[],[]
71077,1096,fbq9ht8l,09/06/2020,Peer-Review,2019 (COVID-19) present range devastating acut...,['il6'],[],"[COVID-19, acute, clinical manifestation, lung...",[],[],[IL-6]
72011,2030,k88ny3i4,19/06/2020,Peer-Review,SARS-CoV-2 caused COVID-19 pandemic. urgent ne...,[],[],"[SARS-CoV-2, COVID-19 pandemic, physiological,...",[],[],[]
72625,2644,g0zccwz2,23/06/2020,Peer-Review,SUMMARY: report describes cat initial respirat...,[],[],"[SUMMARY, report, cat, initial, respiratory si...",[],[],[]


In [23]:
#get top n words
def get_top_n_genes(corpus, n=20):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]#Convert most freq words to dataframe for plotting bar plot


In [29]:
#Check if the subselection contains any genes
def check_if_empty(list_of_lists):
    lst =[]

    for elem in list_of_lists:
       
        if len(elem) == 0:
            lst.append(False)
        else:
            lst.append(True)
    if True in lst:
        return True
    else:
        return False
    

#visualises a column of the using altair of the top items in a column
def viz_data(df,col,n=20):
    sr = df[col].dropna()
    lst = sr.tolist()
    check = check_if_empty(lst)
    if check == True:
            flat_list = [item for sublist in lst for item in sublist]
            top_words = get_top_n_genes(lst, n)
            top_df = pd.DataFrame(top_words)
            top_df.columns=[col, "Freq_gene"]
            plot = alt.Chart(top_df).mark_bar().encode(
    x = alt.X(col,sort ='-y', axis=alt.Axis( title = col)),
    y = alt.Y('Freq_gene', axis=alt.Axis( title = 'Frequency Count'))
    ).properties(title = "Frequency of the top {} {}".format(str(n),col), width = 200
               )

            return plot
    else:
        return False
#visualise top genes and drugs 
def viz_multiple_columns(df,lst,n=20):
    plots = []
    for el in lst:
        plt = viz_data(df,el,n)
        if plt != False:
            plots.append(plt)
            #plots.append('|')
    
    a = plots[0]
    for i in range(1,len(plots)):
         a = a | plots[i]
        
    return a

viz_multiple_columns(df,['Genes','Drugs'],10)

alt.HConcatChart(...)

In [20]:
#This function returns a df with the frequency of the top genes or drugs
#for downstream applications

def get_df(df,col,n=20):
    sr = df[col].dropna()
    lst = sr.tolist()
    check = check_if_empty(lst)
    if check == True:
            flat_list = [item for sublist in lst for item in sublist]
            top_words = get_top_n_genes(lst, n)
            top_df = pd.DataFrame(top_words)
            top_df.columns=[col, "Freq_gene"]

            return top_df
    else:
        return False
#Use it for genes or Drugs
top_words_df = get_df(df,'Genes', n=20)
top_words_df

,Genes,Freq_gene
0,ace2,4960
1,il6,1408
2,ppiap10,1184
3,tmprss2,638
4,cd4,562
5,cd8a,544
6,ins,323
7,nhs,299
8,il10,252
9,ereg,243


In [30]:

#visualises a column of the using altair of the top items in a column

def viz_data_up_reg(df,col,n=20):
    sr = df[col].dropna()
    lst = sr.tolist()
    check = check_if_empty(lst)
    if check == True:
            top_words = get_top_n_genes(lst, n)
            top_df = pd.DataFrame(top_words)
            top_df.columns=[col, "Freq_gene"]
            plot = alt.Chart(top_df).mark_bar().encode(
    x = alt.X(col,sort ='-y', axis=alt.Axis( title = col)),
    y = alt.Y('Freq_gene', axis=alt.Axis( title = 'Frequency Count'))
    ).properties(title = "Frequency of the top {} {}".format(str(n),col), width = 200
               )

            return plot
    else:
        return False
    #first we need to check if the lists are empty

#visualise regulation
def viz_multiple_columns_reg(df,lst,n=20):
    plots = []
    for el in lst:
        plt = viz_data_up_reg(df,el,n)
        if plt != False:
            plots.append(plt)
            #plots.append('|')
    
    a = plots[0]
    for i in range(1,len(plots)):
         a = a | plots[i]
        
    return a

viz_multiple_columns_reg(df,['Genes_Upregulated','Genes_Downregulated','Genes_Nonregulated'],10)

alt.HConcatChart(...)

In [100]:
#This function returns a df with the frequency of the regulation
#for downstream applications

def get_df(df,col,n=20):
    sr = df[col].dropna()
    lst = sr.tolist()
    check = check_if_empty(lst)
    if check == True:
            top_words = get_top_n_genes(lst, n)
            top_df = pd.DataFrame(top_words)
            top_df.columns=[col, "Freq_gene"]

            return top_df
    else:
        return False
#Use it for reguraion
top_words_df = get_df(df,'Genes_Upregulated', n=20)
top_words_df

,Genes_Upregulated,Freq_gene
0,ace2,601
1,il,293
2,crp,221
3,cd4,60
4,cd8,49
5,tmprss2,43
6,10,38
7,saa,37
8,albumin,30
9,er,30
